In [1]:
import numpy as np
import pandas as pd
import re
import datetime
from math import sqrt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
RANDOM_STATE = 846

### Подготовка данных

In [2]:
#данные по рейсам
df = pd.read_csv(r'D:\project\flights.csv', low_memory=False)

In [3]:
#отображать все столбцы
pd.set_option('display.max_columns', None)

In [4]:
df.sample(10)

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
1471237,2015,4,5,7,US,1961,N971UY,RDU,CLT,925,920.0,-5.0,15.0,935.0,62.0,60.0,40.0,130,1015.0,5.0,1027,1020.0,-7.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
369685,2015,1,25,7,WN,541,N758SW,LAX,RNO,1140,1134.0,-6.0,9.0,1143.0,80.0,72.0,58.0,391,1241.0,5.0,1300,1246.0,-14.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1465550,2015,4,4,6,OO,5507,N787SK,DEN,ASE,1848,1845.0,-3.0,8.0,1853.0,45.0,40.0,29.0,125,1922.0,3.0,1933,1925.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
5060387,2015,11,12,4,AA,1249,N024AA,ATL,LAX,1835,1912.0,37.0,17.0,1929.0,304.0,292.0,270.0,1947,2059.0,5.0,2039,2104.0,25.0,0,0,NaN,0.0,0.0,1.0,24.0,0.0
3441282,2015,8,2,7,DL,1730,N307DQ,ATL,CHS,2155,2153.0,-2.0,15.0,2208.0,72.0,59.0,38.0,259,2246.0,6.0,2307,2252.0,-15.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2666365,2015,6,17,3,AS,623,N585AS,LAS,SEA,1715,1709.0,-6.0,15.0,1724.0,150.0,148.0,126.0,867,1930.0,7.0,1945,1937.0,-8.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4514793,2015,10,9,5,WN,1813,N439WN,12889,13232,700,656.0,-4.0,14.0,710.0,210.0,217.0,199.0,1521,1229.0,4.0,1230,1233.0,3.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3278361,2015,7,24,5,UA,1935,N19117,LAX,EWR,859,901.0,2.0,16.0,917.0,338.0,317.0,286.0,2454,1703.0,15.0,1737,1718.0,-19.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
935012,2015,3,3,2,WN,963,N415WN,MDW,MSY,935,953.0,18.0,52.0,1045.0,145.0,173.0,117.0,825,1242.0,4.0,1200,1246.0,46.0,0,0,NaN,28.0,0.0,4.0,14.0,0.0
1799197,2015,4,25,6,EV,5356,N754EV,PHF,ATL,1240,1236.0,-4.0,7.0,1243.0,108.0,106.0,89.0,508,1412.0,10.0,1428,1422.0,-6.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#анализ пропусков данных
df.isna().sum()

YEAR                         0
MONTH                        0
DAY                          0
DAY_OF_WEEK                  0
AIRLINE                      0
FLIGHT_NUMBER                0
TAIL_NUMBER              14721
ORIGIN_AIRPORT               0
DESTINATION_AIRPORT          0
SCHEDULED_DEPARTURE          0
DEPARTURE_TIME           86153
DEPARTURE_DELAY          86153
TAXI_OUT                 89047
WHEELS_OFF               89047
SCHEDULED_TIME               6
ELAPSED_TIME            105071
AIR_TIME                105071
DISTANCE                     0
WHEELS_ON                92513
TAXI_IN                  92513
SCHEDULED_ARRIVAL            0
ARRIVAL_TIME             92513
ARRIVAL_DELAY           105071
DIVERTED                     0
CANCELLED                    0
CANCELLATION_REASON    5729195
AIR_SYSTEM_DELAY       4755640
SECURITY_DELAY         4755640
AIRLINE_DELAY          4755640
LATE_AIRCRAFT_DELAY    4755640
WEATHER_DELAY          4755640
dtype: int64

In [6]:
#расшифровка авиакомпаний
airlines_list = pd.read_csv(r'D:\project\airlines.csv')
airlines_list

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [7]:
#в исходных данных часть кодов аэропортов указана в виде числового значения
#заменим данные значения на соответствующий буквенный код
#csv-файл с кодами аэропортов выгружен с https://www.kaggle.com/datasets/smiller933/bts
ap_letter_codes = pd.read_csv(r'D:\project\L_AIRPORT.csv')
ap_numeric_codes = pd.read_csv(r'D:\project\L_AIRPORT_ID.csv')
#соединим таблицы с буквенными и числовыми кодами
ap = ap_letter_codes.merge(ap_numeric_codes, how='outer', on='Description')
ap

,Code_x,Description,Code_y
0,01A,"Afognak Lake, AK: Afognak Lake Airport",10001.0
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip",10003.0
2,04A,"Lik, AK: Lik Mining Camp",10004.0
3,05A,"Little Squaw, AK: Little Squaw Airport",10005.0
4,06A,"Kizhuyak, AK: Kizhuyak Bay",10006.0
...,...,...,...
6494,NaN,"Athens, Greece: Hellinikon",16421.0
6495,NaN,"Loeriesfontein, South Africa: Loeriesfontein A...",16435.0
6496,NaN,"Oslo, Norway: Oslo Metropolitan Area",16437.0
6497,NaN,"Lima, OH: Lima Airport",16438.0


In [8]:
ap['num_code'] = ''
#приведем числовое значение к типу int, пропущенные значения оставляем
for i in range(ap.shape[0]):
    if not ap.Code_y.isna()[i]: ap.loc[i, 'num_code'] = int(ap.Code_y[i].astype('int'))
    else: ap.loc[i, 'num_code'] = ap.Code_y[i]
ap.rename({'Code_x':'letter_code'}, axis=1, inplace=True)
ap.drop('Code_y', axis=1, inplace=True)
ap

,letter_code,Description,num_code
0,01A,"Afognak Lake, AK: Afognak Lake Airport",10001
1,03A,"Granite Mountain, AK: Bear Creek Mining Strip",10003
2,04A,"Lik, AK: Lik Mining Camp",10004
3,05A,"Little Squaw, AK: Little Squaw Airport",10005
4,06A,"Kizhuyak, AK: Kizhuyak Bay",10006
...,...,...,...
6494,NaN,"Athens, Greece: Hellinikon",16421
6495,NaN,"Loeriesfontein, South Africa: Loeriesfontein A...",16435
6496,NaN,"Oslo, Norway: Oslo Metropolitan Area",16437
6497,NaN,"Lima, OH: Lima Airport",16438


In [9]:
%%time
#произведем замену числовых кодов аэропортов в df на текстовые коды
for i in range(df.shape[0]):
    #выбираем для замены данные в виде строк из 5 цифр - числовые обозначения кода аэропортов
    if re.fullmatch('\d{5}', df.ORIGIN_AIRPORT[i]):
        a = int(df.ORIGIN_AIRPORT[i])
        b = ap[ap.num_code == a].index[0]
        df.loc[i, 'ORIGIN_AIRPORT'] = ap.loc[b, 'letter_code']
    if re.fullmatch('\d{5}', df.DESTINATION_AIRPORT[i]):
        a = int(df.DESTINATION_AIRPORT[i])
        b = ap[ap.num_code == a].index[0]
        df.loc[i, 'DESTINATION_AIRPORT'] = ap.loc[b, 'letter_code']

CPU times: total: 9min 47s
Wall time: 9min 47s


In [25]:
#дополнительно был проведен анализ затраченного на данную задачу времени с другим условием
#было выявлено, что использование нижеуказанного подхода на 13,4% замедляет выполнение задачи
'''
for i in range(df.shape[0]):
    try:
        if int(df.ORIGIN_AIRPORT[i]) in ap.num_code.unique():
            a = int(df.ORIGIN_AIRPORT[i])
            b = ap[ap.num_code == a].index[0]
            df.loc[i, 'ORIGIN_AIRPORT'] = ap.loc[b, 'letter_code']
    except Exception: continue
    else: continue
for i in range(df.shape[0]):
    try:
        if int(df.DESTINATION_AIRPORT[i]) in list_of_codes:
            a = int(df.DESTINATION_AIRPORT[i])
            b = ap[ap.num_code == a].index[0]
            df.loc[i, 'DESTINATION_AIRPORT'] = ap.loc[b, 'letter_code']
    except Exception: continue
    else: continue
'''

In [10]:
df.ORIGIN_AIRPORT.unique()

array(['ANC', 'LAX', 'SFO', 'SEA', 'LAS', 'DEN', 'SLC', 'PDX', 'FAI',
       'MSP', 'PHX', 'SJU', 'PBG', 'IAG', 'PSE', 'BQN', 'ORD', 'GEG',
       'HNL', 'ONT', 'MCO', 'BOS', 'HIB', 'ABR', 'MAF', 'DFW', 'MKE',
       'IAH', 'BNA', 'BRO', 'VPS', 'BOI', 'BJI', 'SGF', 'PHL', 'SBN',
       'RDD', 'EUG', 'IAD', 'BUF', 'PWM', 'JFK', 'CRP', 'PIA', 'FAT',
       'SMF', 'AUS', 'MCI', 'ATL', 'JAX', 'MFR', 'IDA', 'MSN', 'DCA',
       'SAT', 'CHS', 'SBA', 'SMX', 'IND', 'CLE', 'GSP', 'BDL', 'ABI',
       'RIC', 'BFL', 'OMA', 'RDM', 'FLL', 'CID', 'TPA', 'SYR', 'ROC',
       'TYR', 'LAN', 'XNA', 'GSO', 'EWR', 'PBI', 'RSW', 'OAK', 'PVD',
       'RNO', 'PIT', 'ABQ', 'MIA', 'BWI', 'LGA', 'TUL', 'LIT', 'MSY',
       'OKC', 'ATW', 'PNS', 'MEM', 'TYS', 'MHT', 'SAV', 'CLT', 'GRB',
       'ABE', 'JAN', 'OAJ', 'FAR', 'ERI', 'LEX', 'CWA', 'MSO', 'TTN',
       'AMA', 'CLL', 'HOU', 'JLN', 'MLI', 'RDU', 'CVG', 'MHK', 'MOB',
       'TLH', 'BHM', 'CAE', 'TXK', 'ACY', 'DTW', 'RAP', 'TUS', 'EAU',
       'DLH', 'FSD',

In [11]:
df.DESTINATION_AIRPORT.unique()

array(['SEA', 'PBI', 'CLT', 'MIA', 'ANC', 'MSP', 'DFW', 'ATL', 'IAH',
       'PDX', 'MCI', 'FLL', 'ORD', 'HNL', 'PHX', 'EWR', 'JFK', 'MCO',
       'BOS', 'BDL', 'ITO', 'SFO', 'KOA', 'OGG', 'MYR', 'DTW', 'LIH',
       'DEN', 'SJU', 'LAX', 'BWI', 'IAD', 'BQN', 'BUF', 'LGA', 'HOU',
       'SLC', 'PHL', 'SJC', 'OAK', 'LGB', 'TPA', 'DCA', 'TTN', 'BTR',
       'LAS', 'RSW', 'BRD', 'STL', 'RKS', 'MBS', 'SNA', 'MEI', 'MDW',
       'SAN', 'RIC', 'AUS', 'OTZ', 'PIT', 'JAX', 'MSY', 'ONT', 'PSP',
       'BUR', 'DAL', 'CVG', 'SMF', 'RDU', 'JMS', 'BNA', 'DSM', 'MAF',
       'BOI', 'ELP', 'TUS', 'SCC', 'HPN', 'STT', 'MDT', 'RHI', 'SBP',
       'MKE', 'JNU', 'CMH', 'CLD', 'KTN', 'CAK', 'CRP', 'CLE', 'GPT',
       'SHV', 'TYS', 'IND', 'LIT', 'SAT', 'SRQ', 'TUL', 'GRK', 'PNS',
       'BTV', 'CHS', 'DAY', 'OKC', 'SAV', 'XNA', 'COS', 'GJT', 'BZN',
       'PUB', 'HRL', 'HDN', 'MEM', 'GEG', 'ORH', 'SYR', 'GSO', 'VPS',
       'LAW', 'ACY', 'LBB', 'JAC', 'BIL', 'EUG', 'ASE', 'TVC', 'MTJ',
       'CAE', 'PVD',

In [12]:
#функция по приведению значений дат-времени в корректный вид для дальнейшего анализа
def time_decoder(df, column, how):
    for i in range(df.shape[0]):
        a = df[column][i]
        #проверка на NaN
        if a != a: continue
        hours = int(a/100)
        minutes = int(a-hours*100)
        #в некоторых столбцах встречается время 24:00 вместо 0:00, корректируем
        if hours == 24: hours = 0
        if how == 1: #создаем объект времени (ЧЧ:ММ)
            time = datetime.time(hours, minutes).strftime("%H:%M")
        elif how == 2: #считаем время как количество минут от 0:00
            time = hours*60+minutes
        else: 
            print('Ошибка! Некорректное значение аргумента "how"')
            return
        df.loc[i, column] = time

### 1. Найти аэропорт с минимальной задержкой вылета

In [13]:
#для задачи потребуются все числовые неотрицательные значения задержки 
#в рамках условия не считаем более ранний вылет как задержку
df1 = df[['ORIGIN_AIRPORT', 'DEPARTURE_DELAY']]
df1

,ORIGIN_AIRPORT,DEPARTURE_DELAY
0,ANC,-11.0
1,LAX,-8.0
2,SFO,-2.0
3,LAX,-5.0
4,SEA,-1.0
...,...,...
5819074,LAX,-4.0
5819075,JFK,-4.0
5819076,JFK,-9.0
5819077,MCO,-6.0


In [14]:
df1.shape

(5819079, 2)

In [15]:
df1.isna().sum()

ORIGIN_AIRPORT         0
DEPARTURE_DELAY    86153
dtype: int64

In [16]:
#т.к. в некоторых записях отсутствует фактическое время отправления, то, соответственно, мы не можем определить 
#фактическую задержку, поэтому в рамках задачи будем пренебрегать данными записями
df1 = df1.dropna(axis=0, how='any')

In [17]:
df1.shape

(5732926, 2)

In [18]:
df1.isna().sum()

ORIGIN_AIRPORT     0
DEPARTURE_DELAY    0
dtype: int64

In [19]:
#очищаем записи, где вылет был произведен раньше запланированного времени
df1 = df1[df1.DEPARTURE_DELAY>=0]

In [20]:
df1.shape

(2454978, 2)

In [21]:
#получим сумму всех задержек в разрезе каждого аэропорта
df1 = df1.groupby(['ORIGIN_AIRPORT']).sum()
df1

,DEPARTURE_DELAY
ORIGIN_AIRPORT,
ABE,30170.0
ABI,26599.0
ABQ,239533.0
ABR,9154.0
ABY,11655.0
...,...
WRG,7869.0
WYS,994.0
XNA,150337.0


In [22]:
#вывод
df1 = df1[df1.DEPARTURE_DELAY == df1.DEPARTURE_DELAY.min()]
df1 = df1.merge(ap, left_on='ORIGIN_AIRPORT', right_on='letter_code')
print(f'аэропорт с минимальной задержкой вылета — "{df1.Description[0]}"\nкод аэропорта  — "{df1.letter_code[0]}" ({df1.num_code[0]})\nсуммарная задержка по всем рейсам за анализируемый период составляет {int(df1.DEPARTURE_DELAY[0])} минут')

аэропорт с минимальной задержкой вылета — "Moab, UT: Canyonlands Field"
код аэропорта  — "CNY" (11092)
суммарная задержка по всем рейсам за анализируемый период составляет 263 минут


### 2. Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport

In [23]:
#для анализа нужно найти идентификатор аэропорта
ap[ap.Description.str.contains('Los Angeles International')]

,letter_code,Description,num_code
2985,LAX,"Los Angeles, CA: Los Angeles International",12892


In [24]:
#отбираем рейсы в Los Angeles International Airport, убираем лишние столбцы
df2 = df[df.DESTINATION_AIRPORT=='LAX'][['AIRLINE', 'ARRIVAL_DELAY']]
df2

,AIRLINE,ARRIVAL_DELAY
101,OO,0.0
103,OO,-16.0
129,OO,56.0
131,OO,NaN
156,UA,4.0
...,...,...
5818936,AA,55.0
5818976,UA,-47.0
5819002,AA,-2.0
5819018,UA,-7.0


In [25]:
df2.isna().sum()

AIRLINE             0
ARRIVAL_DELAY    2652
dtype: int64

In [26]:
#удаляем строки с пустыми значениями задержки прилета
df2.dropna(axis=0, how='any', inplace=True)
df2

,AIRLINE,ARRIVAL_DELAY
101,OO,0.0
103,OO,-16.0
129,OO,56.0
156,UA,4.0
195,UA,-11.0
...,...,...
5818936,AA,55.0
5818976,UA,-47.0
5819002,AA,-2.0
5819018,UA,-7.0


In [27]:
df2.isna().sum()

AIRLINE          0
ARRIVAL_DELAY    0
dtype: int64

In [28]:
#считаем, что самая пунктуальная авиакомпания - та, что прилетает как можно ближе к запланированному времени
#иными словами - та, у которой наименьшее отклонение значений по задержке и раннему прибытию от нуля
#анализ проведем по медиане и среднему значению (чем меньше среднее отклонение от нуля, тем пунктуальнее авиакомпания)
#для целей задачи направление отклонения не имеет значения, поэтому берем значения отклонений по модулю
df2['abs_delay'] = df2['ARRIVAL_DELAY'].abs()
df2 = df2.groupby(['AIRLINE']).agg({'abs_delay':['median', 'mean']})
df2 = df2.abs_delay.sort_values(['median', 'mean']).reset_index()
df2

,AIRLINE,median,mean
0,AS,11.0,15.956182
1,WN,11.0,19.194698
2,VX,11.0,19.212544
3,OO,11.0,19.658367
4,F9,12.0,22.054248
5,HA,13.0,17.304842
6,DL,13.0,20.397294
7,US,14.0,20.400438
8,UA,14.0,23.593996
9,NK,14.0,27.730358


In [29]:
#вывод
print(f"Самой пунктуальной авиакомпанией на прилет в аэропорт 'Los Angeles International Airport' является {airlines_list.AIRLINE[airlines_list.IATA_CODE==df2.loc[0, 'AIRLINE']].values[0]} ({df2.loc[0, 'AIRLINE']}) \nсреднее отклонение от времени прибытия составляет {np.round(df2.loc[0, 'mean'], decimals=2)} минут")

Самой пунктуальной авиакомпанией на прилет в аэропорт 'Los Angeles International Airport' является Alaska Airlines Inc. (AS) 
среднее отклонение от времени прибытия составляет 15.96 минут


### 3. Найти аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение)

In [30]:
#taxi_out - время на рулении при отправлении, taxi_in - по прилете
df3_1 = df[['ORIGIN_AIRPORT', 'TAXI_OUT']]
df3_2 = df[['DESTINATION_AIRPORT', 'TAXI_IN']]

In [31]:
df3_1.isna().sum()

ORIGIN_AIRPORT        0
TAXI_OUT          89047
dtype: int64

In [32]:
df3_2.isna().sum()

DESTINATION_AIRPORT        0
TAXI_IN                92513
dtype: int64

In [33]:
#очищаем от пустых значений
df3_1 = df3_1.dropna(axis=0, how='any')
df3_2 = df3_2.dropna(axis=0, how='any')

In [34]:
df3_1.isna().sum()

ORIGIN_AIRPORT    0
TAXI_OUT          0
dtype: int64

In [35]:
df3_2.isna().sum()

DESTINATION_AIRPORT    0
TAXI_IN                0
dtype: int64

In [36]:
#приводим столбцы к единому наименованию для дальнейшего объединения (по типу union all)
df3_1.rename(columns={'ORIGIN_AIRPORT':'AIRPORT', 'TAXI_OUT':'ROLLING'}, inplace=True)
df3_2.rename(columns={'DESTINATION_AIRPORT':'AIRPORT', 'TAXI_IN':'ROLLING'}, inplace=True)

In [37]:
#объединяем в одну таблицу, группируем по аэропортам, агрегация по среднему значению
df3 = pd.concat([df3_1, df3_2], axis=0)
df3 = df3.groupby(by='AIRPORT').mean().sort_values(by='ROLLING', ascending=False)
df3

,ROLLING
AIRPORT,
LGA,17.788691
JFK,17.745088
ORD,16.365225
EWR,14.879636
CLT,14.645538
...,...
DLG,5.584416
BET,5.463453
ADQ,4.964516


In [38]:
#получаем информацию об искомом аэропорте и данных по нему
max_rolling_ap = df3.ROLLING.max()
longest_ap = df3[df3.ROLLING==max_rolling_ap].reset_index().loc[0,'AIRPORT']
longest_ap_name = ap[ap.letter_code==longest_ap].reset_index().loc[0,'Description']

In [39]:
#вывод
print(f'Аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение) — "{longest_ap_name}" ({longest_ap})\nСреднее время на рулении составляет {np.round(max_rolling_ap, decimals=2)} минут')

Аэропорт, где самолёты проводят больше всего времени на рулении (среднее значение) — "New York, NY: LaGuardia" (LGA)
Среднее время на рулении составляет 17.79 минут


### 4. Построить модель, которая будет выбирать топ-3 аэропорта прилета в зависимости от аэропорта вылета (вероятность опоздания минимальная RMSE метрика)

In [40]:
#корректируем время вылета в минуты (абсолютное значение) для корректного обучения модели
time_decoder(df, 'SCHEDULED_DEPARTURE', 2)

In [41]:
#функция по определению топ-3 аэропорта прилета в зависимости от аэропорта вылета
def choose_destination(origin_airport, dep_time):
    analyzer = {} #словарь для сбора основных данных по каждому аэропорту
    df4 = df[df.ORIGIN_AIRPORT==origin_airport]
    #удаляем отмененные и перенаправленные рейсы (данные рейсы не дают практической пользы для определения наличия задержки и ее длительности)
    df4 = df4[(df4.CANCELLED==0)&(df4.DIVERTED==0)]
    #удалим бесполезные с точки зрения анализа столбцы
    df4.drop(['AIRLINE',
              'DAY_OF_WEEK',
              'FLIGHT_NUMBER',
              'TAIL_NUMBER',
              'DEPARTURE_TIME',
              'YEAR',
              'WHEELS_OFF',
              'SCHEDULED_TIME',
              'ELAPSED_TIME',
              'WHEELS_ON',
              'ARRIVAL_TIME',
              'SCHEDULED_ARRIVAL',
              'DIVERTED',
              'CANCELLED',
              'CANCELLATION_REASON',
              'AIR_SYSTEM_DELAY',
              'SECURITY_DELAY',
              'AIRLINE_DELAY',
              'LATE_AIRCRAFT_DELAY',
              'WEATHER_DELAY'], axis=1, inplace=True)
    #проходим по каждому аэропорту назначения
    for dest_ap in df4.DESTINATION_AIRPORT.unique():
        chars_set = [] #набор характеристик для рейса по выбранным аэропортам
        _df4 = df4[df4.DESTINATION_AIRPORT==dest_ap]
        amount_of_flights = _df4.shape[0] #количество рейсов между данными аэропортами
        #инициируем фильтр - если рейсов сделано не более 100, то анализ по данному аэропорту прибытия не проводить
        if amount_of_flights<=100: continue
        avg_arriv_diver = _df4.ARRIVAL_DELAY.mean() #среднее время отклонения от времени прилета
        delay_expect = _df4.ARRIVAL_DELAY[_df4.ARRIVAL_DELAY>0].count()/_df4.ARRIVAL_DELAY.count() #вероятность опоздания
        distance = _df4.DISTANCE.mode()[0] #дистанция между выбранными аэропортами
        avg_flight_dur = _df4.AIR_TIME.mean() #средняя продолжительность полета
        #обучим модель на предсказание задержки рейса
        _df4 = _df4.drop(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT'], axis=1)
        x_train, x_test, y_train, y_test = train_test_split(_df4.drop('ARRIVAL_DELAY', axis=1),
                                                  _df4.ARRIVAL_DELAY,
                                                  test_size=0.25,
                                                  random_state=RANDOM_STATE)
        model = LinearRegression()
        model.fit(x_train, y_train)
        #предсказание по тестовой выборке
        pred = model.predict(x_test)
        #получаем оценку RMSE по полученным данным и среднее предсказанное значение задержки
        rmse = sqrt(mean_squared_error(y_test, pred))
        pred_mean_arr_delay = pred.mean()
        #упаковываем полученную информацию
        chars_set.extend([avg_arriv_diver, np.round(delay_expect*100, decimals=1), distance, avg_flight_dur, amount_of_flights, pred_mean_arr_delay, rmse])
        analyzer[dest_ap] = chars_set
    cols = ['среднее отклонение прилета', 'вероятность опоздания', 'дистанция', 'время полета', 'количество рейсов', 'предсказанная задержка по ретроданным', 'RMSE']
    #вся полученная информация по каждому аэропорту прибытия собирается в датафрейм для дальнейшего анализа
    res_df = pd.DataFrame.from_dict(analyzer, orient='index', columns=cols)
    res_df = res_df.sort_values(by=['предсказанная задержка по ретроданным']).reset_index()
    res_df.rename({'index':'аэропорт прилета'}, axis=1, inplace=True)
    #вывод
    print(f'Топ-3 аэропорта прибытия, если вылетать из аэропорта {ap.Description[ap.letter_code==origin_airport].values[0]} ({origin_airport}):\n')
    print(f'1. {ap.Description[ap.letter_code==res_df.loc[0, "аэропорт прилета"]].values[0]} ({res_df.loc[0, "аэропорт прилета"]}), ожидаемая задержка — {np.round(res_df.loc[0, "предсказанная задержка по ретроданным"], decimals=2)} минут, вероятность опоздания — {res_df.loc[0, "вероятность опоздания"]}%')
    print(f'2. {ap.Description[ap.letter_code==res_df.loc[1, "аэропорт прилета"]].values[0]} ({res_df.loc[1, "аэропорт прилета"]}), ожидаемая задержка — {np.round(res_df.loc[1, "предсказанная задержка по ретроданным"], decimals=2)} минут, вероятность опоздания — {res_df.loc[1, "вероятность опоздания"]}%')
    print(f'3. {ap.Description[ap.letter_code==res_df.loc[2, "аэропорт прилета"]].values[0]} ({res_df.loc[2, "аэропорт прилета"]}), ожидаемая задержка — {np.round(res_df.loc[2, "предсказанная задержка по ретроданным"], decimals=2)} минут, вероятность опоздания — {res_df.loc[2, "вероятность опоздания"]}%')
    return res_df

In [188]:
#прогон расчета по всем аэропортам отправления для определения средних значений кф корреляции (для формирования выводов)
#
#кф корреляции рассчитаны в функции "choose_destination" путем добавления строки, например: 
#return res_df['среднее отклонение прилета'].corr(res_df['вероятность опоздания'])
#которая в финальной версии функции (после исполнения кода в данном блоке) была удалена как нецелевая
#
#если в аэропорте отправления недостаточно рейсов/аэропортов прибытия для корректного исполнения функции, то возникают ошибки
#данные ошибки не имеют смысла в рамках определения кф корреляции, в таких случаях возвращается значение NaN и добавляется в список
#пренебрегаем ошибками с помощью модуля "warnings"
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    qq = []
    for i in df.ORIGIN_AIRPORT.unique():
        try:
            qq.append(choose_destination(i, 0)) #SFO
        except: pass
qqq = pd.DataFrame(qq, columns=['qwerty'])
qqq.dropna(axis=0, inplace=True)
qqq.qwerty.mean()

0.8292597473740562

In [42]:
choose_destination('SFO', 0)

Топ-3 аэропорта прибытия, если вылетать из аэропорта San Francisco, CA: San Francisco International (SFO):

1. Pittsburgh, PA: Pittsburgh International (PIT), ожидаемая задержка — -6.44 минут, вероятность опоздания — 30.0%
2. Detroit, MI: Detroit Metro Wayne County (DTW), ожидаемая задержка — -3.56 минут, вероятность опоздания — 31.2%
3. Atlanta, GA: Hartsfield-Jackson Atlanta International (ATL), ожидаемая задержка — -3.41 минут, вероятность опоздания — 27.1%


,аэропорт прилета,среднее отклонение прилета,вероятность опоздания,дистанция,время полета,количество рейсов,предсказанная задержка по ретроданным,RMSE
0,PIT,-4.010101,30.0,2254,253.861953,297,-6.438712,5.522472
1,DTW,-2.805517,31.2,2079,238.643448,1450,-3.559840,3.551487
2,ATL,-3.756112,27.1,2139,243.001767,3395,-3.413332,6.624096
3,CLT,-1.943548,35.3,2296,264.281754,1984,-3.379099,6.321469
4,DCA,0.175487,34.8,2442,278.224234,718,-2.908705,5.467284
...,...,...,...,...,...,...,...,...
68,OTH,19.056180,52.4,412,66.228464,267,14.546752,1.356349
69,STL,10.614271,48.2,1735,203.819028,967,15.405267,7.935688
70,OKC,16.069444,60.0,1384,165.236111,360,16.169951,3.715771
71,JAC,18.956790,53.1,737,96.962963,162,18.007173,2.340183


### При решении задачи по предсказанию величины отклонения от запланированного времени прилета (Х) было выявлено следующее:
#### 1. Х сильно коррелирует с вероятностью опоздания (Rср = 70.9%), рассчитанной по ретроданным. При этом корреляция рассчитанных по ретроданным среднего отклонения от времени прилета и вероятности опоздания составляет 82.9%. Вывод: мы можем с большой степенью достоверности в целях решения задачи №4 выбирать аэропорт прилета, опираясь только на вероятность опоздания
#### 2. величина ошибки RMSE не коррелирует с Х (Rср = 10.3%)
#### 3. точность предсказания возрастает (RMSE достигает минимальных значений) при использовании в обучении модели значений по отклонению от времени вылета и длительности "на рулении" (существенное влияние), а также даты и времени отправления, длительности полета, расстояния между аэропортами. Считаю использование данных значений в модели допустимым, т.к. их можно заранее задать на основе ретроданных (например, время руления в прогнозе можно считать как среднее время руления за имеющийся период)
#### 4. наименьшее значение RMSE достигается при использовании тестовой выборки в размере 25% от генеральной